In [29]:
from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor, Speech2TextModel
from scipy import io
import os
import torch
import torch.nn as nn
from collections import OrderedDict

In [30]:
mat_file = 'out_sentence_details_timit_all_loudness.mat'
file_path = '/home/ahmedb/projects/Wav2Letter/data/'
sentences = io.loadmat(os.path.join(file_path, mat_file), struct_as_record = False, squeeze_me = True, )
features = sentences['features']
phn_names = sentences['phnnames']
sentdet = sentences['sentdet']
fs = sentdet[0].soundf 


In [31]:
wav = sentdet[12].sound
wav.shape

(44775,)

In [32]:
# model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
model = Speech2TextModel.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

Some weights of the model checkpoint at facebook/s2t-small-librispeech-asr were not used when initializing Speech2TextModel: ['lm_head.weight']
- This IS expected if you are initializing Speech2TextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Speech2TextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Speech2TextModel were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.encoder.embed_positions.weights', 'model.decoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
input_features = processor(wav, padding=True, sampling_rate=fs, return_tensors="pt").input_features
generated_ids = model(input_features)
processor.batch_decode(generated_ids)

ValueError: too many values to unpack (expected 2)

In [7]:
weights = model.state_dict()


In [8]:
for k, v in weights.items():
    print(k, "\t", v.shape)

model.encoder.conv.conv_layers.0.weight 	 torch.Size([1024, 80, 5])
model.encoder.conv.conv_layers.0.bias 	 torch.Size([1024])
model.encoder.conv.conv_layers.1.weight 	 torch.Size([512, 512, 5])
model.encoder.conv.conv_layers.1.bias 	 torch.Size([512])
model.encoder.embed_positions.weights 	 torch.Size([6002, 256])
model.encoder.layers.0.self_attn.k_proj.weight 	 torch.Size([256, 256])
model.encoder.layers.0.self_attn.k_proj.bias 	 torch.Size([256])
model.encoder.layers.0.self_attn.v_proj.weight 	 torch.Size([256, 256])
model.encoder.layers.0.self_attn.v_proj.bias 	 torch.Size([256])
model.encoder.layers.0.self_attn.q_proj.weight 	 torch.Size([256, 256])
model.encoder.layers.0.self_attn.q_proj.bias 	 torch.Size([256])
model.encoder.layers.0.self_attn.out_proj.weight 	 torch.Size([256, 256])
model.encoder.layers.0.self_attn.out_proj.bias 	 torch.Size([256])
model.encoder.layers.0.self_attn_layer_norm.weight 	 torch.Size([256])
model.encoder.layers.0.self_attn_layer_norm.bias 	 torch.Siz

In [9]:
print(model)

Speech2TextForConditionalGeneration(
  (model): Speech2TextModel(
    (encoder): Speech2TextEncoder(
      (conv): Conv1dSubsampler(
        (conv_layers): ModuleList(
          (0): Conv1d(80, 1024, kernel_size=(5,), stride=(2,), padding=(2,))
          (1): Conv1d(512, 512, kernel_size=(5,), stride=(2,), padding=(2,))
        )
      )
      (embed_positions): Speech2TextSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0): Speech2TextEncoderLayer(
          (self_attn): Speech2TextAttention(
            (k_proj): Linear(in_features=256, out_features=256, bias=True)
            (v_proj): Linear(in_features=256, out_features=256, bias=True)
            (q_proj): Linear(in_features=256, out_features=256, bias=True)
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=256, out_features=2048, bias=True)
        

In [21]:
# modules = list([*model.modules()])
# mdls = OrderedDict([*model.named_modules()])
encoder = mdls['model.encoder']
decoder = mdls['model.decoder']
head = mdls['lm_head']

In [12]:
mdls.keys()

odict_keys(['', 'model', 'model.encoder', 'model.encoder.conv', 'model.encoder.conv.conv_layers', 'model.encoder.conv.conv_layers.0', 'model.encoder.conv.conv_layers.1', 'model.encoder.embed_positions', 'model.encoder.layers', 'model.encoder.layers.0', 'model.encoder.layers.0.self_attn', 'model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.self_attn_layer_norm', 'model.encoder.layers.0.fc1', 'model.encoder.layers.0.fc2', 'model.encoder.layers.0.final_layer_norm', 'model.encoder.layers.1', 'model.encoder.layers.1.self_attn', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.out_proj', 'model.encoder.layers.1.self_attn_layer_norm', 'model.encoder.layers.1.fc1', 'model.encoder.layers.1.fc2', 'model.encoder.layers.1.final_layer_norm', '

In [26]:
copy = nn.Sequential(encoder, decoder, head)

In [17]:
input_features = processor(wav, padding=True, sampling_rate=fs, return_tensors="pt").input_features
generated_ids = model.generate(input_features)
processor.batch_decode(generated_ids)

/home/ahmedb/.local/lib/python3.8/site-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:559: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1


['it had gone like clockwork']

In [27]:
copy(input_features)

ValueError: too many values to unpack (expected 2)

In [2]:
epoch = 10
print(f"Value to be printed is : {epoch:03d}")

Value to be printed is : 010
